In [1]:
# Diagnostics: run this after selecting the kernel "Python (dental-xray)"
import sys, platform
print("Python:", sys.version)
print("Executable:", sys.executable)
print("Platform:", platform.platform())
try:
    from ultralytics import YOLO
    import torch, numpy as np
    print("ultralytics, torch, numpy OK")
    print("Torch:", torch.__version__)
    print("Device:", 'MPS' if torch.backends.mps.is_available() else 'CUDA' if torch.cuda.is_available() else 'CPU')
except Exception as e:
    print("Import error:", e)

Python: 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 11:23:37) [Clang 14.0.6 ]
Executable: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/.venv/bin/python
Platform: macOS-26.0.1-arm64-arm-64bit-Mach-O
ultralytics, torch, numpy OK
Torch: 2.9.0
Device: MPS
ultralytics, torch, numpy OK
Torch: 2.9.0
Device: MPS


# YOLOv8 Training for Dental X-ray Cavity Detection

## About YOLOv8

YOLOv8 is the latest version of the YOLO (You Only Look Once) object detection family, developed by Ultralytics.

### Key Features:

- **Fast and Accurate**: State-of-the-art object detection performance
- **Multiple Variants**: n (nano), s (small), m (medium), l (large), x (extra-large)
- **Easy to Use**: Simple Python API for training and inference
- **Transfer Learning**: Uses pretrained weights for faster convergence

### Model Variants:

- **YOLOv8n** (nano): Fastest, lowest accuracy (~3M parameters)
- **YOLOv8s** (small): Balanced speed/accuracy (~11M parameters)
- **YOLOv8m** (medium): Better accuracy, slower (~26M parameters)
- **YOLOv8l** (large): High accuracy (~44M parameters)
- **YOLOv8x** (extra-large): Best accuracy, slowest (~68M parameters)

## This Notebook

We'll train **YOLOv8n** (nano variant) on our dental X-ray dataset to:

1. Detect cavities in X-ray images
2. Classify cavity severity (4 classes: 0-3)
3. Save the best model weights for deployment


In [2]:
# ============================================================
# SETUP PATHS AND IMPORTS
# ============================================================

from ultralytics import YOLO
from pathlib import Path
import shutil

# Detect project root (works from notebooks/ or project root)
project_root = (Path('..').resolve() if Path.cwd().name == 'notebooks' else Path('.').resolve())

# Define paths
processed = project_root / 'data' / 'processed'
dataset_yaml = processed / 'dataset.yaml'
models_dir = project_root / 'models'
models_dir.mkdir(parents=True, exist_ok=True)

# Verify dataset.yaml exists
if not dataset_yaml.exists():
    raise FileNotFoundError(
        f"dataset.yaml not found at {dataset_yaml}\n"
        "Please run '01_prepare_dataset.ipynb' first!"
    )

print('✓ Project root:', project_root)
print('✓ Dataset config:', dataset_yaml)
print('✓ Models directory:', models_dir)

✓ Project root: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection
✓ Dataset config: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/data/processed/dataset.yaml
✓ Models directory: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/models


## Step 1: PyTorch Compatibility Patch

Fix for PyTorch 2.6+ to allow loading pretrained weights.


In [3]:
# PyTorch 2.6+ compatibility patch
import torch
import warnings
import os

# Suppress warnings
warnings.filterwarnings('ignore')
os.environ['PYTHONWARNINGS'] = 'ignore'

_ORIGINAL_TORCH_LOAD = torch.load

def _patched_load(*args, **kwargs):
    if 'weights_only' not in kwargs:
        kwargs['weights_only'] = False
    return _ORIGINAL_TORCH_LOAD(*args, **kwargs)

torch.load = _patched_load
print("✓ PyTorch patch applied")

✓ PyTorch patch applied


## Step 2: Load YOLOv8 Model

Load pretrained YOLOv8n (nano) model for transfer learning.


In [4]:
# Load YOLOv8n pretrained model
model = YOLO('yolov8n.pt', verbose=False)
print("✓ YOLOv8n loaded")

✓ YOLOv8n loaded


## Step 3: Training Configuration

Set hyperparameters for training.

**Note:** Ultralytics will show its standard training output including:

- Epoch progress with box_loss, cls_loss, dfl_loss metrics
- Validation results after each epoch
- This is normal and shows training is working correctly


In [5]:
# Training configuration
train_config = {
    'data': str(dataset_yaml),
    'epochs': 50,
    'imgsz': 640,
    'device': 'mps',
    'verbose': True,  # Keep verbose for progress bar
    'name': 'yolov8_dental',
    'patience': 10,
    'save': True,
    'plots': True,
}

print(f"Training for {train_config['epochs']} epochs on {train_config['device']}")

Training for 50 epochs on mps


In [6]:
# Train the model
print("Starting training...")
results = model.train(**train_config)
print("Training complete!")


Starting training...
Ultralytics 8.3.225 🚀 Python-3.13.5 torch-2.9.0 MPS (Apple M1)
Ultralytics 8.3.225 🚀 Python-3.13.5 torch-2.9.0 MPS (Apple M1)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/data/processed/dataset.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, na

KeyboardInterrupt: 

## Step 5: Save Model

Copy best weights to models directory.


In [ ]:
# Save best model weights
try:
    latest_run_dir = Path(results.save_dir)
    source_weights = latest_run_dir / 'weights' / 'best.pt'
    dest_weights = models_dir / 'yolov8_best.pt'

    if source_weights.exists():
        shutil.copy(source_weights, dest_weights)
        size_mb = dest_weights.stat().st_size / (1024*1024)
        print(f"✓ Model saved: {dest_weights} ({size_mb:.1f} MB)")
    else:
        print(f"✗ Weights not found at {source_weights}")
        
except NameError:
    print("✗ Run training cell first")



STARTING YOLOV8 TRAINING
Dataset: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/data/processed/dataset.yaml
Epochs: 1
Image Size: 640
Device: mps


TRAINING COMPLETE!

TRAINING COMPLETE!


## Step 6: Test Inference

Test the trained model on a sample image.


In [ ]:
# Test inference on sample image
from PIL import Image
import matplotlib.pyplot as plt
import cv2

trained_model = YOLO(str(dest_weights))

val_images_dir = processed / 'val' / 'images'
test_images = list(val_images_dir.glob('*.png')) + list(val_images_dir.glob('*.jpg'))

if test_images:
    test_img = test_images[0]
    results = trained_model.predict(source=str(test_img), verbose=False)
    
    print(f"Testing on: {test_img.name}")
    print(f"Detected {len(results[0].boxes)} cavities")
    
    # Visualize
    annotated_img = results[0].plot()
    annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(10, 8))
    plt.imshow(annotated_img_rgb)
    plt.axis('off')
    plt.title(f"YOLOv8 Detection - {test_img.name}")
    plt.show()


✗ Warning: Could not find trained weights at /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/runs/detect/yolov8_dental/weights/best.pt

Training logs saved in: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/runs/detect/yolov8_dental
You can find:
  - Training metrics: results.csv
  - Training plots: *.png files (F1 curve, PR curve, confusion matrix, etc.)
  - Model weights: weights/best.pt, weights/last.pt


## Step 6: Test Model on Sample Image

Let's verify the trained model works by testing it on a sample X-ray image.


# Suppress verbose logs

import logging
logging.getLogger("ultralytics").setLevel(logging.ERROR)
